# Trabalho de Particle Swarm Optimization (PSO)

## Problema do Empacotamento
### Bin Packing Problem (BPP)

Dada uma quantidade inteira e positiva de pacotes/depósitos de capacidade $C$ e um set de $M$ itens  $I = [I_1, \cdots, I_M]$ de tamanhos $S = [S_1,\cdots,S_M]$, o problema consiste em empacotar todos os itens nos pacotes, de modo a não exceder a capacidade $C$, **minimizando** a quantidade $N$ de pacotes utilizados.

## Imports

In [1]:
import numpy as np
import copy
#from pyswarm import pso # Provavelmente não vai usar

## Modelando o Problema

### PSO
**A otimização é dada conforme um dos seguintes sistemas de equações:**

$$
\huge{
\left\{
\begin{align}
&x_i(t+1)=x_i(t)+v_i(t+1)\\
&v_i(t+1)=\omega\cdot v_i(t)+c_1\cdot R_1\cdot\{x_i^{pbest}(t)-x_i(t)\}+c_2\cdot R_2\cdot\{x^{gbest}(t)-x_i(t)\}
\end{align}
\right.}
$$

**Ou então**

$$
\huge{
\left\{
\begin{align}
x_i(t+1)=&x_i(t)+v_i(t+1)\\
v_i(t+1)=&\omega\cdot v_i(t)+c_1\cdot R_1\cdot\{x_i^{pbest}(t)-x_i(t)\}\\
&+c_2\cdot R_2\cdot\{x^{gbest}(t)-x_i(t)\}\\
&+c_3\cdot R_3\cdot\{x^{sgbest}(t)-x_i(t)\}
\end{align}
\right.}
$$

**Onde:**

- $x_i(t)$ é um vetor de posições, no instante $t$;
- $v_1(t)$ é um vetor de velocidades, no instante $t$;
- $x_i^{pbest}(t)$ é a melhor posição conhecida da partícula $i$, até o instante $t$;
- $x^{gbest}(t)$ é a melhor posição conhecida do enxame até o instante $t$;
- $x^{sgbest}(t)$ é a segunda melhor posição conhecida do enxame até o instante $t$;
- $\omega,\;c_2,\;c_3\;\in [0,2[$ são coeficientes sócio-cognitivos:
    - $\omega$ é referente à inércia;
        - dado por: $\omega = \omega_{max}-(\omega_{max}-\omega_{min})\cdot\frac{t}{t_{max}}$
    - $c_1$ é referente à influência da melhor posição conhecida da partícula;
    - $c_2$ é referente à influência da melhor posição conhecida do enxame;
    - Onde adota-se $c_1 = c_2 = 1,5$.
    - $c_3$ é referente à influência da segunda melhor posição conhecida do enxame;
        - Adota-se $c_3 = 1,9$.
- $R_1,\;R_2,\;R_3$ são constantes geradas aleatóriamente $\in [0,1[$;

#### Fitness

$$
\huge{
\begin{align}
f_{BPP} = \frac{\sum^{N}_{i=1}(fill_i/C)^K}{N}\\
\end{align}}
$$

$$
\begin{matrix*}
    \text{Seja:}\left(
    \begin{matrix*}
        \textrm{\textbf{N} o número de pacotes (bins),}\\
        \textrm{\textbf{fill} a soma dos tamanhos dos itens no pacote \textbf{i},}\\
        \textrm{\textbf{C} a capacidade do pacote}\\
        \textrm{\textbf{k} uma constante de elitismo, } k\gt1\\
    \end{matrix*}\right)
\end{matrix*}
$$

---

## Implementação

#### Vetor X de posições
$$
\begin{matrix}
x\;\text{ é um vetor de posições, onde cada posição}\\
x_i\;\text{ representa a i-ésima partícula e:}
\end{matrix}\quad
\left\{\begin{align}
&x_{i} = \quad\text{é uma lista de pacotes/bins}\\
&x_{i_j} = \quad\text{é o j-ésimo pacote e contem n tuplas }(int,\; int)\text{ com (índice, valor/peso) do item}\\
\end{align}
\right.
$$

#### Mudança de Posição
Baseada na equação de velocidade

- A inércia ($\omega$) foi adaptada para representar o quanto a posição da partícula se mantém estática;
- Os coeficientes de influência ($c_1,\;c_2,\;c_3$):
    - São ordenados de forma crescente;
    - SE $c_i \gt \omega\quad i\in[1,3]$ ENTÂO:
        - O pacote com o maior valor de $fill_i$ do respectivo $x_i^{pbest},\;x_i^{gbest},\;x_i^{sgbest}$ é incluído em $x_i$;
        - Duplicatas são removidas (para garantir partículas válidas);
    - $\forall \text{ pacote } x_{i_j} | fill_j \le \theta$ (coeficiente de preenchimento):
        - Os itens de $x_{i_j}$ são agrupados e re-inseridos em $x_i$ usando a heurística MBS.

### Heurísticas

In [2]:
def FF(individuo, item, C):

    indv = copy.deepcopy(individuo)

    if indv == []:
        indv.append([item])
        return indv

    added = False
    for pacote in indv:
        if fill(pacote) + item[1] <= C:
            pacote.append(item)
            added = True
            break

    if not added:
        indv.append([item])

    return indv

In [4]:
class BPPPSO():
    def __init__(self,
                 itens: list[(int, int)] = [],
                 C: int = 0,
                 K: float = 0.0,
                 num_part: int = 10,
                 x: list[list[list[(int, int)]]] = [],
                 t: int = 1,
                 t_max: int = 200,
                 x_pb: list[list[list[(int, int)]]] = [],
                 x_gb: list[list[(int, int)]] = [],
                 x_sgb:list[list[(int, int)]] = [],
                 w: float = -1.0,
                 w_max: float = 0.9,
                 w_min: float = 0.4,
                 c1: float = 1.5,
                 c2: float = 1.5,
                 c3: float = 1.9, # 1.5 
                 R1: float = -1.0,
                 R2: float = -1.0,
                 R3: float = -1.0):
        self.itens = itens
        self.C = C
        self.K = K
        self.num_part = num_part
        self.x = [[] for _ in range(self.num_part)]
        self.v = v
        self.t = t
        self.x_pb = [[] for _ in range(self.num_part)]
        self.x_gb = x_gb
        self.x_sgb = x_sgb
        self.w = w
        self.w_max = w_max
        self.w_min = w_min
        self.c1 = c1
        self.c2 = c2
        self.c3 = c3
        self.R1 = R1
        self.R2 = R2
        self.R3 = R3

    def fill(self, pacote: list[(int, int)]) -> int:
        return sum(x[1] for x in pacote)

    def fitness(self, individuo: list[list[(int, int)]]) -> float:
        soma = 0
        for pacote in individuo:
            soma += (fill(pacote)/self.C) ** self.K
            
        return soma/len(individuo)

    def inicializa(self) -> None:
        lista_itens = []
        for i in range(0, self.num_part):
            lista_itens = copy.deepcopy(this.itens)
            np.random.shuffle(lista_itens)
            for item in lista_itens:
                self.x[i] = FF(self.x[i], item, self.C)

    def itera(self) -> None:
        pass

    def main(self) -> None:
        pass

## Testes/Execução